In [ ]:
! python -m spacy download en_core_web_sm

In [4]:
import pickle
import random
from tqdm import tqdm

import spacy
from spacy.tokens import DocBin

for dev_set in ['train', 'val', 'test']:

    with open(f'{dev_set}.pkl', 'rb') as f:

        data = pickle.load(f)

    random.shuffle(data)

    nlp = spacy.load("en_core_web_sm") # load other spacy model

    db = DocBin() # create a DocBin object

    unrecon = []

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        annot["entities"] = [list(x) for x in set([tuple(x) for x in annot["entities"]])]
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print(text)
                print("Skipping entity")
            else:
                ents.append(span)
        try:
            doc.ents = ents # label the text with the ents
        except:
            unrecon.append(text)
        db.add(doc)

    db.to_disk(f"../data/{dev_set}.spacy")

 36%|███▌      | 2055/5764 [00:00<00:00, 4119.63it/s]

which TV Show where made in  and South Korea?
Skipping entity
Present all Movie crafted by McG.
Skipping entity


100%|██████████| 849/849 [00:00<00:00, 3845.04it/s]
